In [2]:
%run ../../Utils/yp_utils.py

# Initial setup

In [3]:
paper_pmid = 31799324
paper_name = 'lage_mendes_ferreira_2019' 

In [4]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [5]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [6]:
original_data = pd.read_excel('raw_data/Euroscarf 5200 genes.xlsx', sheet_name='Susceptibility to SO2')

In [7]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 767 x 5


In [8]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [10]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Gene, Susceptibility to SO2, Unnamed: 3, Unnamed: 4]
Index: []


In [12]:
t = [-len(s.strip()) for s in original_data['Susceptibility to SO2']]

In [13]:
original_data['data'] = t

In [14]:
original_data.set_index('ORF', inplace=True)
original_data.index.name='orf'

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(767, 2)

# Load & process tested strains

In [17]:
tested = pd.read_excel('raw_data/Euroscarf 5200 genes.xlsx', sheet_name='Euroscarf (5200)')

In [18]:
tested['ORF'] = tested['ORF'].astype(str)
tested['ORF'] = clean_orf(tested['ORF'])

In [19]:
tested['ORF'] = translate_sc(tested['ORF'], to='orf')

In [20]:
tested.loc[tested['ORF']=='YLF023W','ORF'] = 'YFL023W'

In [21]:
# Make sure everything translated ok
t = looks_like_orf(tested['ORF'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [ORF, Gene]
Index: []


In [22]:
tested_orfs = np.unique(tested['ORF'].values)

In [23]:
missing = [t for t in original_data.index.values if t not in tested_orfs]
missing

['YKL006W']

In [24]:
tested_orfs = np.append(tested_orfs, missing)

# Prepare the final dataset

In [25]:
data = original_data.reindex(index=tested_orfs, fill_value=0)[['data']]

In [26]:
dataset_ids = [16405]
datasets = datasets.reindex(index=dataset_ids)

In [27]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [28]:
print('Final data dimensions: %d x %d' % (data.shape))

Final data dimensions: 4945 x 1


In [29]:
data.sum()

dataset_id  data_type
16405       value       -1049
dtype: int64

## Subset to the genes currently in SGD

In [30]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [31]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [32]:
data.head()

,dataset_id,16405
,data_type,value
gene_id,orf,
2,YAL002W,-2
1863,YAL004W,0
4,YAL005C,0
5,YAL007C,0
6,YAL008W,0


# Normalize

In [33]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [34]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [35]:
data_norm[data.isnull()] = np.nan

In [36]:
data_all = data.join(data_norm)

In [37]:
data_all.head()

dataset_id      16405          
data_type       value    valuez
gene_id orf                    
2       YAL002W    -2 -3.497669
1863    YAL004W     0  0.000000
4       YAL005C     0  0.000000
5       YAL007C     0  0.000000
6       YAL008W     0  0.000000

# Print out

In [38]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [43]:
# from IO.save_data_to_db3 import *

In [44]:
# save_data_to_db(data_all, paper_pmid)